# 1. Wikipedia 문서 분석을 위한 기본 데이터 분석

In [2]:
# 라이브러리 불러오기 
import os
import re
import json

import pandas as pd

In [3]:
wiki_path = '../data/wikipedia_documents.json'
wiki_doc = json.load(open(wiki_path, 'r'))

print(f'wiki_doc 길이 : {len(wiki_doc)}')

wiki_doc 길이 : 60613


In [5]:
wiki_doc['0']

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '나라 목록',
 'author': None,
 'html': None,
 'document_id': 0}

In [4]:
wiki_df = pd.DataFrame(wiki_doc).transpose()

wiki_df

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4
...,...,...,...,...,...,...,...,...
60608,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60608
60609,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60609
60610,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60610
60611,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,위키피디아,None,None,단결권 및 단체교섭권 협약,None,None,60611


In [15]:
wiki_df.columns

Index(['text', 'corpus_source', 'url', 'domain', 'title', 'author', 'html',
       'document_id'],
      dtype='object')

In [9]:
wiki_text_list = list(wiki_df['text'].apply(len))

wiki_df['text'].apply(len).describe()

count    60613.000000
mean       755.565044
std        762.962671
min        184.000000
25%        414.000000
50%        577.000000
75%        857.000000
max      46099.000000
Name: text, dtype: float64

In [16]:
# 길이가 5000자를 넘는 문서
print(f"길이가 5000자를 넘는 문서의 개수: {(wiki_df['text'].apply(len) > 5000).sum()}")

길이가 5000자를 넘는 문서의 개수: 163


<br><br>

---

<br><br>

# 2. Wikipedia 문서에서 text가 5천자 이상인 문서에 대한 처리

- 글이 5천자 이상일 경우 반으로 나누어 5천자 미만으로 바꿈 (163개 문서 대상)

In [29]:
wiki_df[wiki_df['text'].apply(len) >= 20000] #46099]

,text,corpus_source,url,domain,title,author,html,document_id
973,:참고 문헌 목록은 제1차 세계 대전의 서지 문서를 참조하십시오.\n \n* tit...,위키피디아,TODO,None,제1차 세계 대전,None,None,973
4543,"인륜을 키미니 례예 둘재라.\n\n○ 孟子曰, 設爲庠序學校, 以敎之. 皆所以明人...",위키피디아,TODO,None,소학언해,None,None,4543
4547,"아다온 말이니 례예 다재라\n\n○ 橫渠張先生曰, 敎小兒, 先要安詳恭敬. 今...",위키피디아,TODO,None,소학언해,None,None,4547
4548,"어딘 실이니 례예 여재라\n\n○ 呂滎公名希哲, 字原明. 申國正獻公之長子. 正...",위키피디아,TODO,None,소학언해,None,None,4548
5527,:참고 문헌 목록은 제1차 세계 대전의 서지 문서를 참조하십시오.\n \n* tit...,위키피디아,None,None,제1차 세계 대전,None,None,5527
9097,"인륜을 키미니 례예 둘재라.\n\n○ 孟子曰, 設爲庠序學校, 以敎之. 皆所以明人...",위키피디아,None,None,소학언해,None,None,9097
9101,"아다온 말이니 례예 다재라\n\n○ 橫渠張先生曰, 敎小兒, 先要安詳恭敬. 今...",위키피디아,None,None,소학언해,None,None,9101
9102,"어딘 실이니 례예 여재라\n\n○ 呂滎公名希哲, 字原明. 申國正獻公之長子. 正...",위키피디아,None,None,소학언해,None,None,9102
9756,1907년 이후 하와이에 건너가 살던 조선인 노동자들은 하와이 농장에서 사탕수수와 ...,위키피디아,None,None,사진 신부,None,None,9756
14722,김일성의 정체를 싸고 진위논란이 있으나 ‘북한학’을 전공한 서옥식은 그의 저서 ‘거...,위키피디아,None,None,서옥식,None,None,14722


In [24]:
wiki_df[~wiki_df['text'].str.contains('\n')]

,text,corpus_source,url,domain,title,author,html,document_id
49,오클라호마주 털사 태생으로 시카고 대학에서 철학을 전공하였다. 그는 유희수학(rec...,위키피디아,TODO,None,마틴 가드너,None,None,49
55,사람들이 직접 만나서 나누는 대화와 같이 인터넷이나 온라인서비스에 접속한 이용자끼리...,위키피디아,TODO,None,채팅,None,None,55
93,TCP/IP는 패킷 통신 방식의 인터넷 프로토콜인 IP (인터넷 프로토콜)와 전송 ...,위키피디아,TODO,None,인터넷 프로토콜 스위트,None,None,93
96,스페인에서 기턴은 비우엘라(vihuela)로 변형되었다. 비우엘라는 평평한 악기로써...,위키피디아,TODO,None,기타,None,None,96
97,15세기 초반에는 4줄의 복현 르네상스 기타가 출연했다. 이 기타는 깃대 플렉트럼으...,위키피디아,TODO,None,기타,None,None,97
...,...,...,...,...,...,...,...,...
60603,"재래식 전력과 비대칭 전력, 즉 핵까지 갖고 있다한 들, 독자적으로 GPS(GNSS...",위키피디아,None,None,자주국방론,None,None,60603
60604,"소련은 1960년 10월, 알제리를 독립 정부로 최초로 인정해준 나라였다. 소련이 ...",위키피디아,None,None,자주국방론,None,None,60604
60606,"곰 자체는 페름 지역 숲에 사는 동물이 풍부함을 상징하며, 은색인 이유는 금속, 소...",위키피디아,None,None,페름의 문장,None,None,60606
60607,"1967년 소련 통치기 페름 시 의회는 새 문장 공모전을 개최하였으며, 1968년까...",위키피디아,None,None,페름의 문장,None,None,60607


In [27]:
wiki_df['text'].iloc[4548]

'어딘 \uf55b실이니 \uf341례예 여\ue991재라\n\n○ 呂滎公名希哲, 字原明. 申國正獻公之長子. 正獻公居家, 簡 重寡黙, 不以事物經心, 而申國夫人, 性嚴有法度, 雖甚愛公, 然, 敎公, 事事循蹈規矩.\n\n呂榮公의 일홈\uee8a 希哲이오 字\ue285 原明이니 申國 正獻公 [일홈 \uee8a 公著ㅣ니 申은 봉\uf53a \uea71히오 正獻은 시회니 宋 적 졍승이 라] 의 \ue564아\ue38f이라 正獻公이 집의 이실 제 간략\uf537며 후듕\uf537며 잡일 아니\uf537며 잡말 아니\uf537야 일과 온갓 거스로\ue73b \ue560\uee93애 경 영티 아니\uf537고 申國夫人이 性이 嚴\uf537고 法度ㅣ 이셔 비록 심 히 公을 \ue97d랑\uf537나 그러나 公을 \ue1a7\ue46c츄\ue3a8 \ue57b\ue97d\ue470 規矩\ue470 조차 \ue653드듸게 \uf537더라\n\n甫十歲, 祁寒暑雨, 侍立終日, 不命之坐, 不敢坐也. 日必冠帶, 以見長者, 平居雖甚熱, 在父母長者之側, 不得去巾襪縳袴, 衣 服唯謹.\n\n계오 열 설에 큰 치위와 덥고 비올 제라두 뫼셔 셧기를 날이 \ue570도록 \uf537야 命\uf537야 안\uf1fc라 아니커든 敢히 안\ueaed 아니\uf537더라 날마다 반\ue38a시 冠帶\uf537야 \ue73b 얼우신\uea45 뵈\uee88오며 샹해 이실 제 비록 심히 더우나 父母와 얼우신 겯틔 이셔 시러곰 곳갈와 보 션과 \uf55b뎐\uee8c 밧디 아니\uf537야 衣服을 오직 삼가더라\n\n行步出入, 無得入茶肆酒肆, 市井里巷之語, 鄭衛之音, 未嘗一 經於耳, 不正之書, 非禮之色, 未嘗一接於目.\n\n\uf537녀 거러 나며 들음애 시러곰 차 \uf48e\ue285 집과 술 \uf48e\ue285 집의 드 디 아니\uf537며 져제와 \ue560\uee8c\uf55a 말와 鄭과 衛ㅅ 소\ue487 [두 나라 일 홈이니 음탕\uf53a 풍\ue264라] 를 일즉 \uf53a번 귀예 디내디 

In [63]:
# 한글 비율을 계산하는 함수
def korean_ratio(text):
    korean_chars = re.findall(r'[가-힣]', text)
    total_chars = len(text)
    
    if total_chars == 0:
        return 0.0  # 텍스트가 비어 있으면 비율 0.0 반환
    
    return len(korean_chars) / total_chars  # 한글 비율 계산

In [68]:
# wiki_df의 text 길이를 max_len 기준 이하로 나누는 함수

# 기준 길이 (max_len 이상이면 글 나누기)
max_len = 5000


# text = wiki_df['text'].iloc[4548]
text = wiki_df['text'].iloc[0]

#print(text)
print(f'\n\n length = {len(text)}')

# 만약 max_len을 넘을 때 동작
def split_text_by_max_len(text, max_len=5000, cut_text_based_enter=True):    
    # 문장 구분 기준 설정 (\n을 기준으로 할지, .?!을 기준으로 할지)
    if cut_text_based_enter and '\n' in text:
        sentences = text.split('\n')
    else:
        sentences = re.split(r'(?<=[.!?])\s+', text)
    
    split_texts = []
    current_text = []

    # 문장을 추가하면서 길이가 max_len을 넘는 경우 나눔
    for sentence in sentences:
        sentence = sentence.strip()  # 공백 제거
        if not sentence:  # 빈 문자열은 건너뜀
            continue

        if len(' '.join(current_text)) + len(sentence) + 1 <= max_len:
            current_text.append(sentence)
        else:
            split_texts.append(' '.join(current_text).strip())
            current_text = [sentence]
    
    # 마지막으로 남은 문장 처리
    if current_text:
        split_texts.append(' '.join(current_text).strip())
    
    return split_texts # list type output


def preprocess_json(wiki_doc, without_sohak=True, max_len=5000, cut_text_based_enter=True, ratio=0.5):
    cut_text_based_enter=True
    
    # wiki json file을 dataframe으로
    df = pd.DataFrame(wiki_doc).transpose()
    
    # 소학언해 제외
    if without_sohak:
        df = df[df['title']!='소학언해']
    
    # 각 텍스트의 한글 비율을 계산
    df['korean_ratio'] = df['text'].apply(korean_ratio)    
    df = df[df['korean_ratio'] > ratio]
    df = df.drop(columns=['korean_ratio'])
    

    
    return df
        
    # new_rows = []
    
    # for idx, row in df.itterrows():
    #     text = row['text']
        
    #     # 기준 길이(max_len) 보다 긴 경우
    #     if len(text) > max_len:
    #         split_texts = split_text_by_max_len(text, max_len, cut_text_based_enter)
#
# # 나뉜 텍스트 각각에 대해 새로운 행을 추가
#             for split_text in split_texts:
#                 new_row = row.copy()  # 기존 행을 복사
#                 new_row['text'] = split_text  # 텍스트만 나눠진 것으로 변경
#                 new_rows.append(new_row)  # 새로운 행을 리스트에 추가
#         else:
#             new_rows.append(row)  # 5000자 이하인 경우 원래 행 추가
    
#     # 나눠진 행들로 새로운 DataFrame 생성
#     new_df = pd.DataFrame(new_rows)
    
#     return new_df

# # 나눠진 텍스트를 반영한 새로운 데이터프레임 생성
# wiki_df_split = split_long_texts(wiki_df)

# # 결과 확인
# print(wiki_df_split)
                            
            
        
# test_output = split_text_by_max_len(text, cut_text_based_enter=True)

# print('\n\n================================\n\n test_output = ', len(test_output))
# print('test_output[0] = ',len(test_output[0]))
# print('test_output[0] -> ', test_output[0])

preprocess_json(wiki_doc, without_sohak=True, max_len=5000, cut_text_based_enter=True, ratio=0.2)



 length = 454
0 60613
1 60593
2, ratio보다 작거나 같은 751
3 59842


,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4
...,...,...,...,...,...,...,...,...
60608,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60608
60609,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60609
60610,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60610
60611,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,위키피디아,None,None,단결권 및 단체교섭권 협약,None,None,60611


In [46]:
wiki_df['text'].iloc[4548]

'어딘 \uf55b실이니 \uf341례예 여\ue991재라\n\n○ 呂滎公名希哲, 字原明. 申國正獻公之長子. 正獻公居家, 簡 重寡黙, 不以事物經心, 而申國夫人, 性嚴有法度, 雖甚愛公, 然, 敎公, 事事循蹈規矩.\n\n呂榮公의 일홈\uee8a 希哲이오 字\ue285 原明이니 申國 正獻公 [일홈 \uee8a 公著ㅣ니 申은 봉\uf53a \uea71히오 正獻은 시회니 宋 적 졍승이 라] 의 \ue564아\ue38f이라 正獻公이 집의 이실 제 간략\uf537며 후듕\uf537며 잡일 아니\uf537며 잡말 아니\uf537야 일과 온갓 거스로\ue73b \ue560\uee93애 경 영티 아니\uf537고 申國夫人이 性이 嚴\uf537고 法度ㅣ 이셔 비록 심 히 公을 \ue97d랑\uf537나 그러나 公을 \ue1a7\ue46c츄\ue3a8 \ue57b\ue97d\ue470 規矩\ue470 조차 \ue653드듸게 \uf537더라\n\n甫十歲, 祁寒暑雨, 侍立終日, 不命之坐, 不敢坐也. 日必冠帶, 以見長者, 平居雖甚熱, 在父母長者之側, 不得去巾襪縳袴, 衣 服唯謹.\n\n계오 열 설에 큰 치위와 덥고 비올 제라두 뫼셔 셧기를 날이 \ue570도록 \uf537야 命\uf537야 안\uf1fc라 아니커든 敢히 안\ueaed 아니\uf537더라 날마다 반\ue38a시 冠帶\uf537야 \ue73b 얼우신\uea45 뵈\uee88오며 샹해 이실 제 비록 심히 더우나 父母와 얼우신 겯틔 이셔 시러곰 곳갈와 보 션과 \uf55b뎐\uee8c 밧디 아니\uf537야 衣服을 오직 삼가더라\n\n行步出入, 無得入茶肆酒肆, 市井里巷之語, 鄭衛之音, 未嘗一 經於耳, 不正之書, 非禮之色, 未嘗一接於目.\n\n\uf537녀 거러 나며 들음애 시러곰 차 \uf48e\ue285 집과 술 \uf48e\ue285 집의 드 디 아니\uf537며 져제와 \ue560\uee8c\uf55a 말와 鄭과 衛ㅅ 소\ue487 [두 나라 일 홈이니 음탕\uf53a 풍\ue264라] 를 일즉 \uf53a번 귀예 디내디 

In [43]:
test_output

['',
 '어딘 \uf55b실이니 \uf341례예 여\ue991재라',
 '○ 呂滎公名希哲, 字原明. 申國正獻公之長子. 正獻公居家, 簡 重寡黙, 不以事物經心, 而申國夫人, 性嚴有法度, 雖甚愛公, 然, 敎公, 事事循蹈規矩.',
 '呂榮公의 일홈\uee8a 希哲이오 字\ue285 原明이니 申國 正獻公 [일홈 \uee8a 公著ㅣ니 申은 봉\uf53a \uea71히오 正獻은 시회니 宋 적 졍승이 라] 의 \ue564아\ue38f이라 正獻公이 집의 이실 제 간략\uf537며 후듕\uf537며 잡일 아니\uf537며 잡말 아니\uf537야 일과 온갓 거스로\ue73b \ue560\uee93애 경 영티 아니\uf537고 申國夫人이 性이 嚴\uf537고 法度ㅣ 이셔 비록 심 히 公을 \ue97d랑\uf537나 그러나 公을 \ue1a7\ue46c츄\ue3a8 \ue57b\ue97d\ue470 規矩\ue470 조차 \ue653드듸게 \uf537더라',
 '甫十歲, 祁寒暑雨, 侍立終日, 不命之坐, 不敢坐也. 日必冠帶, 以見長者, 平居雖甚熱, 在父母長者之側, 不得去巾襪縳袴, 衣 服唯謹.',
 '계오 열 설에 큰 치위와 덥고 비올 제라두 뫼셔 셧기를 날이 \ue570도록 \uf537야 命\uf537야 안\uf1fc라 아니커든 敢히 안\ueaed 아니\uf537더라 날마다 반\ue38a시 冠帶\uf537야 \ue73b 얼우신\uea45 뵈\uee88오며 샹해 이실 제 비록 심히 더우나 父母와 얼우신 겯틔 이셔 시러곰 곳갈와 보 션과 \uf55b뎐\uee8c 밧디 아니\uf537야 衣服을 오직 삼가더라',
 '行步出入, 無得入茶肆酒肆, 市井里巷之語, 鄭衛之音, 未嘗一 經於耳, 不正之書, 非禮之色, 未嘗一接於目.',
 '\uf537녀 거러 나며 들음애 시러곰 차 \uf48e\ue285 집과 술 \uf48e\ue285 집의 드 디 아니\uf537며 져제와 \ue560\uee8c\uf55a 말와 鄭과 衛ㅅ 소\ue487 [두 나라 일 홈이니 음탕\uf53a 풍\ue264라] 를 일즉 \uf

In [32]:
def split_text(text):
    if '\n' in text:
        # '\n'이 있을 경우 이를 기준으로 분할
        sentences = text.split('\n')
    else:
        # '\n'이 없으면 문장 단위로 분할
        sentences = re.split(r'(?<=[.!?])\s+', text)
    
    return sentences

# 예시 사용
text_with_newline = "Hello!\nHow are you?\nI'm fine."
text_without_newline = "Hello! How are you? I'm fine."

# 함수 적용
split_with_newline = split_text(text_with_newline)
split_without_newline = split_text(text_without_newline)

print("With newline:", split_with_newline)
print("Without newline:", split_without_newline)


With newline: ['Hello!', 'How are you?', "I'm fine."]
Without newline: ['Hello!', 'How are you?', "I'm fine."]


<br><br>

---

<br><br>

# 3. Wikipedia 문서의 한글 비율을 대상으로 문서 처리


In [ ]:
import re

# 한글이 ratio 이상인지 판단하는 함수
def is_korean_dominant(text):
    korean_chars = re.findall(r'[가-힣]', text)
    total_chars = len(text)
    
    if total_chars == 0:
        return False  # 텍스트가 비어 있으면 False 반환
    
    korean_ratio = len(korean_chars) / total_chars
    return korean_ratio >= 0.1

# 한글이 ratio% 미만인 텍스트 필터링
filtered_df = wiki_df[~wiki_df['text'].apply(is_korean_dominant)]

# 결과 출력
print(filtered_df)


In [ ]:
print(f'\n\n 한글 비율이 30% 미만인 문서 수 : {len(pd.unique(filtered_df.title))} / 총 문서 수 {len(wiki_df)}')
print(f'\n\n한글 비율이 30% 미만인 문서의 title : {pd.unique(filtered_df.title)}')

In [ ]:
# 한글 비율을 계산하는 함수
def korean_ratio(text):
    korean_chars = re.findall(r'[가-힣]', text)
    total_chars = len(text)
    
    if total_chars == 0:
        return 0.0  # 텍스트가 비어 있으면 비율 0.0 반환
    
    return len(korean_chars) / total_chars  # 한글 비율 계산

# 비율 구간을 설정
bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
labels = [f'{int(b*100)}-{int((b+0.1)*100)}%' for b in bins[:-1]]

# 각 텍스트의 한글 비율을 계산
wiki_df['korean_ratio'] = wiki_df['text'].apply(korean_ratio)

# 비율 구간을 기준으로 그룹화
wiki_df['korean_range'] = pd.cut(wiki_df['korean_ratio'], bins=bins, labels=labels, right=False)

# 새로운 DataFrame 생성: 그룹별 데이터 수와 document_id 리스트를 도출
result_df = wiki_df.groupby('korean_range').agg(count=('document_id', 'size'), id_list=('document_id', list)).reset_index()

# 불필요한 열 제거 (원하는 경우)
result_df = result_df.drop(columns=['korean_range'])

# 결과 출력
print(result_df)


In [ ]:
print(wiki_df[wiki_df['document_id'] == 0]['text'].values[0])